In [1]:
import os
import sys
import gensim
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from gensim.models.doc2vec import LabeledSentence
from sklearn.cross_validation import train_test_split
from gensim.models.word2vec import Word2Vec
from tqdm import tqdm
from sklearn import utils
import numpy as np
from keras import optimizers
from keras.models import load_model
from keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, GRU, Bidirectional

C:\Users\User\Anaconda3\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\Users\User\Anaconda3\lib\site-packages\pattern-2.6-py3.6.egg\pattern\text\en\..\..\..\..\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using 

In [2]:
def preprocess(text):
    text = re.sub(r'[^\w\s]','',text)
    text = re.sub(r'\d+','<number>',text)
    return text

In [3]:
clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/train.csv")
# clean_train_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_train_full.csv")
clean_train_comments['content'] = clean_train_comments['content'].apply(preprocess)
clean_train_comments.head()

,content,polarity
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral
2,kasus toko obat digerebek fpi propam akan peri...,neutral
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral
4,minyak jarak castor oil <number> ml,neutral


In [4]:
tokenizer = RegexpTokenizer(r'\w+')
clean_train_comments['content'] = clean_train_comments['content'].astype('str') 
clean_train_comments.dtypes
clean_train_comments['tokens'] = clean_train_comments['content'].apply(tokenizer.tokenize)
clean_train_comments['sentiment'] = clean_train_comments['polarity'].astype('category').cat.codes
   
clean_train_comments.head()

,content,polarity,tokens,sentiment
0,gerindra alihkan rekomendasi ke agus an tanri ...,neutral,"[gerindra, alihkan, rekomendasi, ke, agus, an,...",1
1,cuci tangan pakai sunlight stelah itu pakai sa...,neutral,"[cuci, tangan, pakai, sunlight, stelah, itu, p...",1
2,kasus toko obat digerebek fpi propam akan peri...,neutral,"[kasus, toko, obat, digerebek, fpi, propam, ak...",1
3,menkeu melemah nya rupiah lebih berpengaruh pa...,neutral,"[menkeu, melemah, nya, rupiah, lebih, berpenga...",1
4,minyak jarak castor oil <number> ml,neutral,"[minyak, jarak, castor, oil, number, ml]",1


In [5]:
clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/test.csv")
# clean_test_comments = pd.read_csv("./corpus/prosa/data_clean_punctuation/data_testing_full.csv")
clean_test_comments['content'] = clean_test_comments['content'].apply(preprocess)
clean_test_comments.head()

,content,polarity
0,kemarin gue datang ke tempat makan baru yang a...,negative
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative


In [6]:
clean_test_comments['content'] = clean_test_comments['content'].astype('str') 
clean_test_comments.dtypes
clean_test_comments["tokens"] = clean_test_comments["content"].apply(tokenizer.tokenize)
clean_test_comments['sentiment'] = clean_test_comments['polarity'].astype('category').cat.codes

clean_test_comments.head()

,content,polarity,tokens,sentiment
0,kemarin gue datang ke tempat makan baru yang a...,negative,"[kemarin, gue, datang, ke, tempat, makan, baru...",0
1,kayak nya sih gue tidak akan mau balik lagi ke...,negative,"[kayak, nya, sih, gue, tidak, akan, mau, balik...",0
2,kalau dipikirpikir sebenarnya tidak ada yang b...,negative,"[kalau, dipikirpikir, sebenarnya, tidak, ada, ...",0
3,ini pertama kalinya gua ke bank buat ngurusin ...,negative,"[ini, pertama, kalinya, gua, ke, bank, buat, n...",0
4,waktu sampai dengan gue pernah disuruh ibu lat...,negative,"[waktu, sampai, dengan, gue, pernah, disuruh, ...",0


In [7]:
x_train = clean_train_comments['content']
x_validation = clean_test_comments['content']
y_train = clean_train_comments['sentiment']
y_validation = clean_test_comments['sentiment']

In [8]:
def labelize_text(text,label):
    result = []
    prefix = label
    for i, t in zip(text.index, text):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result
  
x_train = labelize_text(x_train, 'TRAIN')
x_validation = labelize_text(x_validation, 'TEST')

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [9]:
MAX_SEQUENCE_LENGTH = 85
data_dim = 500

In [10]:
word2vec = Word2Vec.load('./prosa-w2v/prosa.vec')
# word2vec = Word2Vec.load("./vectorizer/tripadvisor/word2vec_300.model")

In [11]:
def build_Word_Vector(tokens, size):
    vec = np.zeros((MAX_SEQUENCE_LENGTH - len(tokens), size))
    for word in tokens:
        try:
            vec = np.append(vec, word2vec[word])
        except KeyError: 
            vec = np.append(vec, np.zeros((1, size)))
            continue
    vec.reshape(MAX_SEQUENCE_LENGTH, size)
    return vec

In [12]:
train_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_train))])
val_vecs = np.concatenate([[build_Word_Vector(z, 500)] for z in tqdm(map(lambda x: x.words, x_validation))])

0it [00:00, ?it/s]C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """
935it [00:01, 692.31it/s] 
100it [00:00, 849.99it/s]


In [13]:
batch_size = 256
num_epochs = 10
hidden_size = 10
timesteps = MAX_SEQUENCE_LENGTH
num_class = 1

In [14]:
num_data = len(train_vecs)
num_data_val = len(val_vecs)

train_vecs = train_vecs.reshape((num_data, timesteps, data_dim))
y_train = y_train.reshape((num_data, num_class))
val_vecs = val_vecs.reshape((num_data_val, timesteps, data_dim))
y_validation = y_validation.reshape((num_data_val, num_class))

C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  """
C:\Users\User\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [15]:
model = Sequential()
model.add(Bidirectional(GRU(hidden_size, input_shape=(timesteps, data_dim)), merge_mode='concat'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(train_vecs, to_categorical(y_train), epochs=num_epochs, validation_data=(val_vecs, to_categorical(y_validation)))

Train on 935 samples, validate on 100 samples
Epoch 1/10
935/935 [==============================] - ETA: 1:00 - loss: 1.1344 - acc: 0.250 - ETA: 31s - loss: 1.1448 - acc: 0.359 - ETA: 21s - loss: 1.1386 - acc: 0.36 - ETA: 16s - loss: 1.1422 - acc: 0.36 - ETA: 13s - loss: 1.1607 - acc: 0.33 - ETA: 11s - loss: 1.1382 - acc: 0.36 - ETA: 9s - loss: 1.1579 - acc: 0.3527 - ETA: 8s - loss: 1.1507 - acc: 0.347 - ETA: 7s - loss: 1.1465 - acc: 0.357 - ETA: 6s - loss: 1.1357 - acc: 0.359 - ETA: 6s - loss: 1.1437 - acc: 0.349 - ETA: 5s - loss: 1.1528 - acc: 0.341 - ETA: 5s - loss: 1.1564 - acc: 0.336 - ETA: 4s - loss: 1.1448 - acc: 0.339 - ETA: 4s - loss: 1.1436 - acc: 0.347 - ETA: 3s - loss: 1.1381 - acc: 0.355 - ETA: 3s - loss: 1.1328 - acc: 0.358 - ETA: 3s - loss: 1.1364 - acc: 0.354 - ETA: 2s - loss: 1.1265 - acc: 0.361 - ETA: 2s - loss: 1.1207 - acc: 0.370 - ETA: 2s - loss: 1.1191 - acc: 0.378 - ETA: 1s - loss: 1.1158 - acc: 0.379 - ETA: 1s - loss: 1.1130 - acc: 0.381 - ETA: 1s - loss: 1.1081

935/935 [==============================] - ETA: 4s - loss: 0.6579 - acc: 0.750 - ETA: 4s - loss: 0.6851 - acc: 0.750 - ETA: 4s - loss: 0.6828 - acc: 0.729 - ETA: 3s - loss: 0.6574 - acc: 0.742 - ETA: 3s - loss: 0.6463 - acc: 0.750 - ETA: 3s - loss: 0.6453 - acc: 0.770 - ETA: 3s - loss: 0.6448 - acc: 0.767 - ETA: 3s - loss: 0.6434 - acc: 0.765 - ETA: 3s - loss: 0.6317 - acc: 0.777 - ETA: 3s - loss: 0.6348 - acc: 0.775 - ETA: 2s - loss: 0.6219 - acc: 0.781 - ETA: 2s - loss: 0.6262 - acc: 0.773 - ETA: 2s - loss: 0.6254 - acc: 0.769 - ETA: 2s - loss: 0.6228 - acc: 0.772 - ETA: 2s - loss: 0.6114 - acc: 0.779 - ETA: 2s - loss: 0.6095 - acc: 0.779 - ETA: 1s - loss: 0.6067 - acc: 0.783 - ETA: 1s - loss: 0.6063 - acc: 0.784 - ETA: 1s - loss: 0.6012 - acc: 0.787 - ETA: 1s - loss: 0.5996 - acc: 0.784 - ETA: 1s - loss: 0.6001 - acc: 0.781 - ETA: 1s - loss: 0.6010 - acc: 0.782 - ETA: 0s - loss: 0.6030 - acc: 0.782 - ETA: 0s - loss: 0.6053 - acc: 0.781 - ETA: 0s - loss: 0.6068 - acc: 0.781 - ETA: 0s

In [16]:
# model.save('./model/bi_gru_3/bi_gru_model_01.h5')  

In [17]:
# model = load_model('./model/bi_gru_3/bi_gru_model_01.h5')
y_pred = model.predict(val_vecs)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(y_validation, y_pred, labels = [0, 1, 2], digits=8))

             precision    recall  f1-score   support

          0  0.57142857 0.70000000 0.62921348        40
          1  0.40000000 0.30000000 0.34285714        20
          2  0.58333333 0.52500000 0.55263158        40

avg / total  0.54190476 0.55000000 0.54130945       100

